In [227]:
import numpy as np
!pip install pymorphy2
!pip install distool
!python -m spacy download ru_core_news_md


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 10.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [228]:
import re

from collections import defaultdict
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np
from distool.feature_extraction.symptom_collection import SymptomCollection
from distool.feature_extraction import SymptomExtractor
from distool.feature_extraction.anamnesis import Anamnesis
from distool.feature_extraction.symptom_status import SymptomStatus
import pymorphy2

import pandas as pd

# tqdm.pandas()

In [229]:
marked_df = pd.read_csv("../data/patients_symptoms_marked.csv").set_index("Идентификатор")
marked_df.head()

,Кардиология,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
Идентификатор,,,,,,,,,,,,,,,,,,,,,
b13c995673fb409fb66b5f81c180642b,слабость,1,повышенное артериальное давление,1.0,бледность кожи,1.0,повышение аппетита,1.0,снижение аппетита,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29ceffdca8bf494f9d21b55b4bc2995c,боль в груди,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2c75d3c2b87c400098f12aa710b3dde9,повышенное артериальное давление,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d1129e9686df442b9dee566985735dba,учащенное сердцебиение,1,интоксикация,1.0,боль в груди,1.0,слабый пульс,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
091ae4f1b0d14c82885ccdcdd81af6e3,хроническая усталость,1,боль в груди,1.0,отеки,1.0,поверхностный сон,1.0,повышенное артериальное давление,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
len(marked_df)

17

In [231]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

def try_to_transform_symptom_to_id(symptom_str):
    symptom_id = symptom_str.lower()
    symptom_id = re.sub(r'(\<(/?[^>]+)>)', ' ', symptom_id)
    symptom_id = symptom_id.replace('[^\w\s]',' ')
    symptom_id = re.sub(r'\d+', ' ', symptom_id)
    symptom_id = lemmatize(symptom_id)
    symptom_id = ' '.join(symptom_id)
    return symptom_id

# Find unknown symptoms


In [232]:
existed_symptoms = SymptomCollection.get_symptoms()
len(existed_symptoms)
existed_symptoms

[Symptom(id_name='боль в горло', patterns=[[{'LEMMA': 'боль'}, {'LEMMA': 'в'}, {'LEMMA': 'горло'}]]),
 Symptom(id_name='температура', patterns=[[{'LEMMA': 'температура'}]]),
 Symptom(id_name='вялость', patterns=[[{'LEMMA': 'вялость'}]]),
 Symptom(id_name='головной боль', patterns=[[{'LEMMA': 'головной'}, {'LEMMA': 'боль'}]]),
 Symptom(id_name='увеличить лимфатический узел', patterns=[[{'LEMMA': 'увеличить'}, {'LEMMA': 'лимфатический'}, {'LEMMA': 'узел'}]]),
 Symptom(id_name='снижение аппетит', patterns=[[{'LEMMA': 'снижение'}, {'LEMMA': 'аппетит'}]]),
 Symptom(id_name='повышенный слюнотечение', patterns=[[{'LEMMA': 'повышенный'}, {'LEMMA': 'слюнотечение'}]]),
 Symptom(id_name='боль в ушах', patterns=[[{'LEMMA': 'боль'}, {'LEMMA': 'в'}, {'LEMMA': 'ушах'}]]),
 Symptom(id_name='увеличить миндалина', patterns=[[{'LEMMA': 'увеличить'}, {'LEMMA': 'миндалина'}]]),
 Symptom(id_name='белый налёт на миндалинах', patterns=[[{'LEMMA': 'белый'}, {'LEMMA': 'налёт'}, {'LEMMA': 'на'}, {'LEMMA': 'минда

In [233]:
existed_symptoms_id = list(map(lambda x: x.id_name, existed_symptoms))
existed_symptoms_id

['боль в горло',
 'температура',
 'вялость',
 'головной боль',
 'увеличить лимфатический узел',
 'снижение аппетит',
 'повышенный слюнотечение',
 'боль в ушах',
 'увеличить миндалина',
 'белый налёт на миндалинах',
 'боль в грудь',
 'нарушение прохождение пища',
 'дискомфорт в время еда',
 'кашель',
 'осиплость голос',
 'боль в спина',
 'боль в нога',
 'боль вокруг пупок',
 'тахикардия',
 'головокружение',
 'одышка',
 'учащённый дыхание',
 'обморок',
 'потеря сознание',
 'инфаркт',
 'рассеянность внимание',
 'снижение работоспособность',
 'нарушение память',
 'утомляемость',
 'нарушение сон',
 'снижение интеллект',
 'наружение мозговой кровообращение',
 'инсульт',
 'тромбоз',
 'хромота',
 'похолодание конечность',
 'слабый пульс',
 'синюшность',
 'поверхностный сон',
 'малый длительность сон',
 'сонливость',
 'эмоциональный расстройство',
 'зуд в глаз',
 'ощущение тяжесть веко',
 'переутомление глаз',
 'светобоязнь',
 'отёчность веко',
 'чешуйка на кожа веко',
 'выпадение ресница',
 'н

In [234]:
marked_symptoms = set()
for row_tuple in marked_df.itertuples(index=False, name=None):
    filtered_row = list(filter(lambda x: not pd.isna(x) and isinstance(x, str), row_tuple))
    marked_symptoms.update(filtered_row)

print(marked_symptoms)

{'диарея', 'аллергические реакции', 'потеря сознания', 'зевота', 'повышенное артериальное давление', 'онемение', 'учащенный пульс', 'учащенное сердцебиение', 'атеросклероз', 'похолодание конечностей', 'отеки', 'повышенный холестерин', 'сухой кашель', 'ком в горле', 'хроническая усталость', 'лишний вес', 'гипотериоз', 'боль в печени', 'слабость', 'увеличенные лимфатические узлы', 'утомляемость', 'слабый пульс', 'грыжи позвоночника', 'повышение аппетита', 'поверхностный сон', 'звон в ушах', 'чувство нехватки воздуха', 'боль в груди', 'увеличенная печень', 'синяки под глазами', 'интоксикация', 'панические атаки', 'боль в глазах', 'боль в горле', 'стоматит', 'головная боль', 'сухость кожи', 'боль в суставах', 'одышка', 'чувство страха', 'боль в сердце', 'температура', 'снижение аппетита', 'головокружение', 'геморрой', 'повышенная потливость', 'расслаивание ногтей', 'бледность кожи', 'пониженное артериальное давление'}


In [235]:
marked_symptom_ids = set()
for marked_symptom in marked_symptoms:
    marked_symptom_ids.add(try_to_transform_symptom_to_id(marked_symptom))
print(len(marked_symptom_ids))
print(marked_symptom_ids)

49
{'диарея', 'зевота', 'расслаивание ноготь', 'потеря сознание', 'учащённый пульс', 'боль в печень', 'онемение', 'панический атака', 'учащённый сердцебиение', 'грыжа позвоночник', 'чувство страх', 'атеросклероз', 'похолодание конечность', 'повышенный потливость', 'аллергический реакция', 'бледность кожа', 'повышенный холестерин', 'сухой кашель', 'лишний вес', 'увеличить лимфатический узел', 'слабость', 'боль в грудь', 'снижение аппетит', 'повышенный артериальный давление', 'утомляемость', 'слабый пульс', 'боль в горло', 'поверхностный сон', 'синяк под глаз', 'отечь', 'чувство нехватка воздух', 'интоксикация', 'стоматит', 'сухость кожа', 'боль в сустав', 'хронический усталость', 'повышение аппетит', 'одышка', 'понизить артериальный давление', 'боль в сердце', 'головной боль', 'температура', 'звон в ухо', 'головокружение', 'боль в глаз', 'кто в горло', 'геморрой', 'увеличить печень', 'гипотериоз'}


In [236]:
print(len(marked_symptom_ids.difference(existed_symptoms_id)))
marked_symptom_ids.difference(existed_symptoms_id)

17


{'геморрой',
 'гипотериоз',
 'грыжа позвоночник',
 'звон в ухо',
 'зевота',
 'кто в горло',
 'лишний вес',
 'отечь',
 'панический атака',
 'повышенный артериальный давление',
 'повышенный потливость',
 'повышенный холестерин',
 'понизить артериальный давление',
 'расслаивание ноготь',
 'слабость',
 'стоматит',
 'учащённый пульс'}

# Perform converting dataframe
Need to convert marked dataset to format:
`ID - <list of marked symptom column> - <list of extractor symptom column>`

In [237]:
df_showcase = pd.DataFrame(list(marked_df.index), columns=["case_id"])
df_showcase.head()

,case_id
0,b13c995673fb409fb66b5f81c180642b
1,29ceffdca8bf494f9d21b55b4bc2995c
2,2c75d3c2b87c400098f12aa710b3dde9
3,d1129e9686df442b9dee566985735dba
4,091ae4f1b0d14c82885ccdcdd81af6e3


In [238]:
# Add case column
patient_df = pd.read_csv("../data/patient_case.csv").dropna()
patient_df.rename(columns={"id": "case_id"}, inplace=True)
patient_df.head()

,case_id,name,case,url
0,53b49cb2f8f841e894058b023f0c215a,Акушерство - Беременность-Роды,"<div class=""postcolor"" id=""post-40777"">\n\t\t\...",https://meduniver.com/forum/index.php?s=63007a...
1,2a8a4ae843ed470bbc5cc4b16300417a,Акушерство - Беременность-Роды,"<div class=""postcolor"" id=""post-40637"">\n\t\t\...",https://meduniver.com/forum/index.php?s=63007a...
2,fe560282eee14778bceaf3533a028d4f,Акушерство - Беременность-Роды,"<div class=""postcolor"" id=""post-40361"">\n\t\t\...",https://meduniver.com/forum/index.php?s=63007a...
3,80743a55b8a5493ebfcf48053e45e955,Акушерство - Беременность-Роды,"<div class=""postcolor"" id=""post-40309"">\n\t\t\...",https://meduniver.com/forum/index.php?s=63007a...
4,01a62e5678b4410593c206b0c25d71c4,Акушерство - Беременность-Роды,"<div class=""postcolor"" id=""post-24901"">\n\t\t\...",https://meduniver.com/forum/index.php?s=63007a...


In [239]:
def preprocess_case(df):
    df.case = df.case.str.lower()
    df.case = df.case.apply(lambda x: re.sub(r'(\<(/?[^>]+)>)', ' ', x).strip())
    df.case = df.case.str.replace('[^\w\s]',' ')
    df.case = df.case.apply(lambda x: re.sub(r'\d+', ' ', x))
    df.case = df.case.apply(lambda x: lemmatize(x))
    df.case = df.case.apply(lambda x: ' '.join(x))

In [240]:
preprocess_case(patient_df)

/tmp/ipykernel_25450/4129127589.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.case = df.case.str.replace('[^\w\s]',' ')


In [241]:
patient_df.head()

,case_id,name,case,url
0,53b49cb2f8f841e894058b023f0c215a,Акушерство - Беременность-Роды,здравствуйте на день цикл появиться бледно роз...,https://meduniver.com/forum/index.php?s=63007a...
1,2a8a4ae843ed470bbc5cc4b16300417a,Акушерство - Беременность-Роды,по узи поставить диагноз хронический двухсторо...,https://meduniver.com/forum/index.php?s=63007a...
2,fe560282eee14778bceaf3533a028d4f,Акушерство - Беременность-Роды,беременность неделя беременность первый ребёно...,https://meduniver.com/forum/index.php?s=63007a...
3,80743a55b8a5493ebfcf48053e45e955,Акушерство - Беременность-Роды,здравствуйте пойти на приём к гинеколог из за ...,https://meduniver.com/forum/index.php?s=63007a...
4,01a62e5678b4410593c206b0c25d71c4,Акушерство - Беременность-Роды,здравствуйте задержка день но при это тест отр...,https://meduniver.com/forum/index.php?s=63007a...


In [242]:
patient_df = patient_df.drop(columns=["name", "url"])

In [243]:
df_showcase = df_showcase.join(patient_df.set_index("case_id"), on="case_id")

In [244]:
df_showcase.head()

,case_id,case
0,b13c995673fb409fb66b5f81c180642b,добрый день интересный и запутать ситуация с м...
1,29ceffdca8bf494f9d21b55b4bc2995c,добрый день подсказать пожалуйста муж стать бе...
2,2c75d3c2b87c400098f12aa710b3dde9,уважаемый доктор по результам описание наш док...
3,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...
4,091ae4f1b0d14c82885ccdcdd81af6e3,здравствуйте расшифровать пожалуйста исследова...


# Fill columns with marked symptoms


In [245]:
def symptom_id_to_marked_column_name(id):
    return "marked_" + id.replace(' ', '_')

In [246]:
# Create columns for existed symptom ids
marked_column_existed_symptoms_id = []
for id in existed_symptoms_id:
    column_id = symptom_id_to_marked_column_name(id)
    marked_column_existed_symptoms_id.append(column_id)

In [274]:
marked_column_existed_symptoms_id

['marked_боль_в_горло',
 'marked_температура',
 'marked_вялость',
 'marked_головной_боль',
 'marked_увеличить_лимфатический_узел',
 'marked_снижение_аппетит',
 'marked_повышенный_слюнотечение',
 'marked_боль_в_ушах',
 'marked_увеличить_миндалина',
 'marked_белый_налёт_на_миндалинах',
 'marked_боль_в_грудь',
 'marked_нарушение_прохождение_пища',
 'marked_дискомфорт_в_время_еда',
 'marked_кашель',
 'marked_осиплость_голос',
 'marked_боль_в_спина',
 'marked_боль_в_нога',
 'marked_боль_вокруг_пупок',
 'marked_тахикардия',
 'marked_головокружение',
 'marked_одышка',
 'marked_учащённый_дыхание',
 'marked_обморок',
 'marked_потеря_сознание',
 'marked_инфаркт',
 'marked_рассеянность_внимание',
 'marked_снижение_работоспособность',
 'marked_нарушение_память',
 'marked_утомляемость',
 'marked_нарушение_сон',
 'marked_снижение_интеллект',
 'marked_наружение_мозговой_кровообращение',
 'marked_инсульт',
 'marked_тромбоз',
 'marked_хромота',
 'marked_похолодание_конечность',
 'marked_слабый_пульс',


In [247]:
zipped_rows = []
for row_tuple in marked_df.itertuples(index=False, name=None):
    filtered_row = list(filter(lambda x: not pd.isna(x), row_tuple))
    row_iter = iter(filtered_row)
    zipped_row = list(zip(row_iter, row_iter))
    zipped_rows.append(zipped_row)
print(zipped_rows)

[[('слабость', 1), ('повышенное артериальное давление', 1.0), ('бледность кожи', 1.0), ('повышение аппетита', 1.0), ('снижение аппетита', 1.0), ('увеличенная печень', 1.0), ('геморрой', -1.0), ('повышенный холестерин', 1.0)], [('боль в груди', 1)], [('повышенное артериальное давление', 1)], [('учащенное сердцебиение', 1), ('интоксикация', 1.0), ('боль в груди', 1.0), ('слабый пульс', 1.0)], [('хроническая усталость', 1), ('боль в груди', 1.0), ('отеки', 1.0), ('поверхностный сон', 1.0), ('повышенное артериальное давление', 1.0)], [('учащенное сердцебиение', 1), ('панические атаки', 1.0), ('ком в горле', 1.0)], [('лишний вес', -1), ('повышенное артериальное давление', -1.0), ('атеросклероз', 1.0), ('боль в груди', 1.0), ('ком в горле', 1.0), ('диарея', 1.0), ('аллергические реакции', 1.0)], [('учащенное сердцебиение', 1), ('боль в груди', 1.0), ('утомляемость', 1.0), ('синяки под глазами', 1.0)], [('боль в груди', 1), ('головокружение', 1.0), ('одышка', 1.0), ('похолодание конечностей',

In [260]:
column_symptoms_marks = []
for zipped_row in zipped_rows:
    column_symptoms_marks_row = [SymptomStatus.NO_INFO.value for i in marked_column_existed_symptoms_id] # 3 - No Info status
    for (id, mark) in zipped_row:
        marked_column_name = symptom_id_to_marked_column_name(id)
        print(marked_column_name)
        if marked_column_name in marked_column_existed_symptoms_id:
            print(mark)
            if int(mark) == 1:
               column_symptoms_marks_row[marked_column_existed_symptoms_id.index(marked_column_name)] = SymptomStatus.YES.value
            elif int(mark) == -1:
               column_symptoms_marks_row[marked_column_existed_symptoms_id.index(marked_column_name)] = SymptomStatus.NO.value
    column_symptoms_marks.append(column_symptoms_marks_row)

marked_слабость
marked_повышенное_артериальное_давление
marked_бледность_кожи
marked_повышение_аппетита
marked_снижение_аппетита
marked_увеличенная_печень
marked_геморрой
marked_повышенный_холестерин
marked_боль_в_груди
marked_повышенное_артериальное_давление
marked_учащенное_сердцебиение
marked_интоксикация
1.0
marked_боль_в_груди
marked_слабый_пульс
1.0
marked_хроническая_усталость
marked_боль_в_груди
marked_отеки
marked_поверхностный_сон
1.0
marked_повышенное_артериальное_давление
marked_учащенное_сердцебиение
marked_панические_атаки
marked_ком_в_горле
marked_лишний_вес
marked_повышенное_артериальное_давление
marked_атеросклероз
1.0
marked_боль_в_груди
marked_ком_в_горле
marked_диарея
1.0
marked_аллергические_реакции
marked_учащенное_сердцебиение
marked_боль_в_груди
marked_утомляемость
1.0
marked_синяки_под_глазами
marked_боль_в_груди
marked_головокружение
1.0
marked_одышка
1.0
marked_похолодание_конечностей
marked_поверхностный_сон
1.0
marked_звон_в_ушах
1.0
marked_боль_в_сердце
1


!!! Need to check that we didn't mess marks with ids

In [261]:
df_showcase.loc[:, marked_column_existed_symptoms_id] = column_symptoms_marks
df_showcase.head(100)

/tmp/ipykernel_25450/842095127.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_showcase.loc[:, marked_column_existed_symptoms_id] = column_symptoms_marks


,case_id,case,marked_боль_в_горло,marked_температура,marked_вялость,marked_головной_боль,marked_увеличить_лимфатический_узел,marked_снижение_аппетит,marked_повышенный_слюнотечение,marked_боль_в_ушах,...,marked_боль_в_мышца,marked_шум_в_сердце,marked_синдром_барабанный_палочка,marked_зуд_в_область_анальный_отверстие,marked_скрежет_зуб,marked_боль_в_правый_подреберье,marked_аллергический_реакция,marked_желтуха,marked_отдышка,marked_дискомфорт
0,b13c995673fb409fb66b5f81c180642b,добрый день интересный и запутать ситуация с м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,29ceffdca8bf494f9d21b55b4bc2995c,добрый день подсказать пожалуйста муж стать бе...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,2c75d3c2b87c400098f12aa710b3dde9,уважаемый доктор по результам описание наш док...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,091ae4f1b0d14c82885ccdcdd81af6e3,здравствуйте расшифровать пожалуйста исследова...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,a612ac9ef20747cfa9dc76f5f8cf8283,добрый день помочь пожалуйста разобраться запу...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
7,e3a29b06768e41059a03a364b1e7f9bd,я год переболеть ковид в январь в февраль резк...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
8,85ad093778194cdfb3a1b238c5a0c184,здравствуйте такой проблема пациент молодой го...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
9,cbbf4976d04f4647997764e6a5f8658b,помочь пожалуйста разобраться я год в детский ...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [262]:
df_showcase.head()

,case_id,case,marked_боль_в_горло,marked_температура,marked_вялость,marked_головной_боль,marked_увеличить_лимфатический_узел,marked_снижение_аппетит,marked_повышенный_слюнотечение,marked_боль_в_ушах,...,marked_боль_в_мышца,marked_шум_в_сердце,marked_синдром_барабанный_палочка,marked_зуд_в_область_анальный_отверстие,marked_скрежет_зуб,marked_боль_в_правый_подреберье,marked_аллергический_реакция,marked_желтуха,marked_отдышка,marked_дискомфорт
0,b13c995673fb409fb66b5f81c180642b,добрый день интересный и запутать ситуация с м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,29ceffdca8bf494f9d21b55b4bc2995c,добрый день подсказать пожалуйста муж стать бе...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,2c75d3c2b87c400098f12aa710b3dde9,уважаемый доктор по результам описание наш док...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,091ae4f1b0d14c82885ccdcdd81af6e3,здравствуйте расшифровать пожалуйста исследова...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [263]:
df_showcase.sum()

case_id                            b13c995673fb409fb66b5f81c180642b29ceffdca8bf49...
case                               добрый день интересный и запутать ситуация с м...
marked_боль_в_горло                                                               51
marked_температура                                                                50
marked_вялость                                                                    51
                                                         ...                        
marked_боль_в_правый_подреберье                                                   51
marked_аллергический_реакция                                                      51
marked_желтуха                                                                    51
marked_отдышка                                                                    51
marked_дискомфорт                                                                 51
Length: 373, dtype: object

# Add marks from symptom extractor

In [264]:
def symptom_id_to_extractor_column_name(id):
    return "extractor_" + id.replace(' ', '_')

In [265]:
extractor = SymptomExtractor()

In [266]:
transformed_symptoms = extractor.transform(df_showcase.case)

In [267]:
transformed_symptoms

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 1, 3]])

In [268]:
len(transformed_symptoms[0])

371

In [269]:
# Create columns for extractor symptom ids
extractor_column_existed_symptoms_id = []
for id in existed_symptoms_id:
    column_id = symptom_id_to_extractor_column_name(id)
    extractor_column_existed_symptoms_id.append(column_id)

In [270]:
extractor_column_existed_symptoms_id

['extractor_боль_в_горло',
 'extractor_температура',
 'extractor_вялость',
 'extractor_головной_боль',
 'extractor_увеличить_лимфатический_узел',
 'extractor_снижение_аппетит',
 'extractor_повышенный_слюнотечение',
 'extractor_боль_в_ушах',
 'extractor_увеличить_миндалина',
 'extractor_белый_налёт_на_миндалинах',
 'extractor_боль_в_грудь',
 'extractor_нарушение_прохождение_пища',
 'extractor_дискомфорт_в_время_еда',
 'extractor_кашель',
 'extractor_осиплость_голос',
 'extractor_боль_в_спина',
 'extractor_боль_в_нога',
 'extractor_боль_вокруг_пупок',
 'extractor_тахикардия',
 'extractor_головокружение',
 'extractor_одышка',
 'extractor_учащённый_дыхание',
 'extractor_обморок',
 'extractor_потеря_сознание',
 'extractor_инфаркт',
 'extractor_рассеянность_внимание',
 'extractor_снижение_работоспособность',
 'extractor_нарушение_память',
 'extractor_утомляемость',
 'extractor_нарушение_сон',
 'extractor_снижение_интеллект',
 'extractor_наружение_мозговой_кровообращение',
 'extractor_инсульт

In [271]:
df_showcase

,case_id,case,marked_боль_в_горло,marked_температура,marked_вялость,marked_головной_боль,marked_увеличить_лимфатический_узел,marked_снижение_аппетит,marked_повышенный_слюнотечение,marked_боль_в_ушах,...,marked_боль_в_мышца,marked_шум_в_сердце,marked_синдром_барабанный_палочка,marked_зуд_в_область_анальный_отверстие,marked_скрежет_зуб,marked_боль_в_правый_подреберье,marked_аллергический_реакция,marked_желтуха,marked_отдышка,marked_дискомфорт
0,b13c995673fb409fb66b5f81c180642b,добрый день интересный и запутать ситуация с м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,29ceffdca8bf494f9d21b55b4bc2995c,добрый день подсказать пожалуйста муж стать бе...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,2c75d3c2b87c400098f12aa710b3dde9,уважаемый доктор по результам описание наш док...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,091ae4f1b0d14c82885ccdcdd81af6e3,здравствуйте расшифровать пожалуйста исследова...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,a612ac9ef20747cfa9dc76f5f8cf8283,добрый день помочь пожалуйста разобраться запу...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
7,e3a29b06768e41059a03a364b1e7f9bd,я год переболеть ковид в январь в февраль резк...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
8,85ad093778194cdfb3a1b238c5a0c184,здравствуйте такой проблема пациент молодой го...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
9,cbbf4976d04f4647997764e6a5f8658b,помочь пожалуйста разобраться я год в детский ...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [272]:
df_showcase.loc[:, extractor_column_existed_symptoms_id] = transformed_symptoms
df_showcase.head(100)

/tmp/ipykernel_25450/2100000169.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_showcase.loc[:, extractor_column_existed_symptoms_id] = transformed_symptoms


,case_id,case,marked_боль_в_горло,marked_температура,marked_вялость,marked_головной_боль,marked_увеличить_лимфатический_узел,marked_снижение_аппетит,marked_повышенный_слюнотечение,marked_боль_в_ушах,...,extractor_боль_в_мышца,extractor_шум_в_сердце,extractor_синдром_барабанный_палочка,extractor_зуд_в_область_анальный_отверстие,extractor_скрежет_зуб,extractor_боль_в_правый_подреберье,extractor_аллергический_реакция,extractor_желтуха,extractor_отдышка,extractor_дискомфорт
0,b13c995673fb409fb66b5f81c180642b,добрый день интересный и запутать ситуация с м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,29ceffdca8bf494f9d21b55b4bc2995c,добрый день подсказать пожалуйста муж стать бе...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,2c75d3c2b87c400098f12aa710b3dde9,уважаемый доктор по результам описание наш док...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,091ae4f1b0d14c82885ccdcdd81af6e3,здравствуйте расшифровать пожалуйста исследова...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,d1129e9686df442b9dee566985735dba,приветствовать вообщий несколько месяц назад м...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,a612ac9ef20747cfa9dc76f5f8cf8283,добрый день помочь пожалуйста разобраться запу...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,2
7,e3a29b06768e41059a03a364b1e7f9bd,я год переболеть ковид в январь в февраль резк...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
8,85ad093778194cdfb3a1b238c5a0c184,здравствуйте такой проблема пациент молодой го...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
9,cbbf4976d04f4647997764e6a5f8658b,помочь пожалуйста разобраться я год в детский ...,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,1


In [273]:
df_showcase.sum()

case_id                               b13c995673fb409fb66b5f81c180642b29ceffdca8bf49...
case                                  добрый день интересный и запутать ситуация с м...
marked_боль_в_горло                                                                  51
marked_температура                                                                   50
marked_вялость                                                                       51
                                                            ...                        
extractor_боль_в_правый_подреберье                                                   51
extractor_аллергический_реакция                                                      51
extractor_желтуха                                                                    51
extractor_отдышка                                                                    48
extractor_дискомфорт                                                                 46
Length: 744, dtype: object

# Here we have showcase and ready to compute any metrics